In [1]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly

! pip install -U tf-nightly-gpu

! pip install tfkerassurgeon


Uninstalling tensorflow-1.14.0rc1:
  Successfully uninstalled tensorflow-1.14.0rc1
     |████████████████████████████████| 394.6MB 57kB/s 
     |████████████████████████████████| 3.2MB 46.7MB/s 
     |████████████████████████████████| 501kB 47.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/60/83/7a/8560c6394f0bc8097d0b8e83d3fa4d5562ac62f7b0730d68ff
Successfully built tfkerassurgeon


In [2]:

import tensorflow as tf

from tensorflow.python.keras import layers
from tensorflow.python.keras import activations
from tensorflow.python.keras import models
from tensorflow.python.keras import backend
from tensorflow.python.keras import callbacks
from tensorflow.examples.tutorials import mnist

import tfkerassurgeon
from tfkerassurgeon import identify
from tfkerassurgeon.operations import delete_channels

print(tf.__version__)


1.14.1-dev20190622


In [0]:

# Set some static values that can be tweaked to experiment
keras_verbosity = 2 # limits the printed output but still gets the Epoch stats
epochs=200 # we'd never reach 200 because we have early stopping
batch_size=128 # tweak this depending on your hardware and Model


In [4]:

# Load dataset (it will automatically download it if needed)
dataset = mnist.input_data.read_data_sets('tempData', one_hot=True, reshape=False)



W0622 20:13:57.683336 139665831700352 deprecation.py:323] From <ipython-input-4-275b5aaa04e1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0622 20:13:57.685291 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0622 20:13:57.686792 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting tempData/train-images-idx3-ubyte.gz


W0622 20:13:58.501961 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0622 20:13:58.507020 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting tempData/train-labels-idx1-ubyte.gz


W0622 20:13:58.902250 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tempData/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tempData/t10k-labels-idx1-ubyte.gz


In [0]:

# Simple reusable shorthand to compile the model, so that we can be sure to use the same optomizer, loss, and metrics
def compile_model(model):
    
    model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])



In [0]:

# method that encapsulates the Models archeteture and construction
def build_model():

    # Create LeNet model
    model = models.Sequential()
    model.add(layers.Conv2D(20,
                     [3, 3],
                     input_shape=[28, 28, 1],
                     activation='relu',
                     name='conv_1'))
    model.add(layers.MaxPool2D())
    model.add(layers.Conv2D(50, [3, 3], activation='relu', name='conv_2'))
    model.add(layers.MaxPool2D())
    model.add(layers.Permute((2, 1, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu', name='dense_1'))
    model.add(layers.Dense(10, activation='softmax', name='dense_2'))

    compile_model(model)

    return model


In [7]:

# a simple method that gets the callbacks for training
def get_callbacks(use_early_stopping = True, use_reduce_lr = True):

    callback_list = []

    if(use_early_stopping):

        callback_list.append(callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=0,
                                             patience=10,
                                             verbose=keras_verbosity,
                                             mode='auto'))

    if(use_reduce_lr):

        callback_list.append(callbacks.ReduceLROnPlateau(monitor='val_loss',
                                            factor=0.1,
                                            patience=5,
                                            verbose=keras_verbosity,
                                            mode='auto',
                                            epsilon=0.0001,
                                            cooldown=0,
                                            min_lr=0))

    return callback_list

# and get the callbacks
callback_list = get_callbacks()

W0622 20:13:59.174589 139665831700352 callbacks.py:1780] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


In [0]:

# Simple reusable shorthand for evaluating the model on the Validation set 
def fit_model(model):
    
    return model.fit(
                    dataset.train.images,
                    dataset.train.labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=keras_verbosity,
                    validation_data=(dataset.validation.images, dataset.validation.labels),
                    callbacks=callback_list)


In [0]:

# Simple reusable shorthand for evaluating the model on the Validation set 
def eval_model(model):

    return model.evaluate(
                        dataset.validation.images, 
                        dataset.validation.labels, 
                        batch_size=batch_size, 
                        verbose=keras_verbosity)


In [0]:







# THIS IS WHERE THE MAGIC HAPPENS!
# This method uses the Keras Surgeon to identify which parts od a layer can be pruned and then deletes them
# Note: it returns the new, pruned model, that was recompiled
def prune_layer(model, layer):
    
    # Get the APOZ (Average Percentage of Zeros) that should identify where we can prune
    apoz = identify.get_apoz(model, layer, dataset.validation.images)

    # Get the Channel Ids that have a high APOZ, which indicates they can be pruned
    high_apoz_channels = identify.high_apoz(apoz)

    # Run the pruning on the Model and get the Pruned (uncompiled) model as a result
    model = delete_channels(model, layer, high_apoz_channels)

    # Recompile the model
    compile_model(model)

    return model

  
  
  
  
  
  
  


In [0]:

# A helper that gets the layer by it's name 
def prune_layer_by_name(model, layer_name):

    # First we get the layer we are working on
    layer = model.get_layer(name=layer_name)
    # Then prune is and return the pruned model
    return prune_layer(model, layer)


In [0]:

# the main function, that runs the training
def main(): 

    
    # build the model
    model = build_model()

    # Initial Train on dataset
    results = fit_model(model)

    # eval and print the results of the training
    loss = eval_model(model)
    print('original model loss:', loss, '\n')
    
    # NOTE: This while true will continue until it ERRORs out because there is no escape condition.
    while True:

        # only prune the Dense layer for this example
        layer_name = 'dense_1'
        # Run the Pruning on the layer
        model = prune_layer_by_name(model, layer_name)

        # eval and print the results of the pruning
        loss = eval_model(model)
        print('model loss after pruning: ', loss, '\n')
        
        # Retrain the model to accomodate for the changes
        results = fit_model(model)

        # eval and print the results of the retraining
        loss = eval_model(model)
        print('model loss after retraining: ', loss, '\n')

        # While TRUE will repeat until an ERROR occurs


In [13]:

# Run the main Method
if __name__ == '__main__':
    main()


W0622 20:13:59.275877 139665831700352 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1624: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0622 20:14:00.337045 139665831700352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 55000 samples, validate on 5000 samples
Epoch 1/200
55000/55000 - 6s - loss: 0.2078 - acc: 0.9395 - val_loss: 0.0760 - val_acc: 0.9770
Epoch 2/200
55000/55000 - 2s - loss: 0.0545 - acc: 0.9833 - val_loss: 0.0448 - val_acc: 0.9878
Epoch 3/200
55000/55000 - 2s - loss: 0.0374 - acc: 0.9884 - val_loss: 0.0357 - val_acc: 0.9904
Epoch 4/200
55000/55000 - 2s - loss: 0.0275 - acc: 0.9914 - val_loss: 0.0322 - val_acc: 0.9898
Epoch 5/200
55000/55000 - 2s - loss: 0.0215 - acc: 0.9931 - val_loss: 0.0434 - val_acc: 0.9874
Epoch 6/200
55000/55000 - 2s - loss: 0.0163 - acc: 0.9944 - val_loss: 0.0308 - val_acc: 0.9914
Epoch 7/200
55000/55000 - 2s - loss: 0.0136 - acc: 0.9955 - val_loss: 0.0317 - val_acc: 0.9902
Epoch 8/200
55000/55000 - 2s - loss: 0.0101 - acc: 0.9969 - val_loss: 0.0336 - val_acc: 0.9902
Epoch 9/200
55000/55000 - 2s - loss: 0.0087 - acc: 0.9972 - val_loss: 0.0352 - val_acc: 0.9918
Epoch 10/200
55000/55000 - 2s - loss: 0.0082 - acc: 0.9972 - val_loss: 0.0409 - val_acc: 0.9906


W0622 20:14:46.153893 139665831700352 deprecation_wrapper.py:118] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:10: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



TypeError: ignored